# LSTM process
No model optimizations

In [74]:
from helpers import dateparse
from helpers import fill_missing
from helpers import create_folder
from numpy import nan

import pandas as pd


Since the datasets contain datapoints with the same timestamp (can be considered index in this case) we can go ahead and merge the datasets by the index.

In [75]:
price_data = pd.read_csv('../price_history/processed_datasets/price_history.csv',
                         index_col="timestamp", parse_dates=['timestamp'], date_parser=dateparse, cache_dates=True)
tweets_data = pd.read_csv('../tweets/processed_datasets/tweets_count.csv', index_col="timestamp",
                          parse_dates=['timestamp'], date_parser=dateparse, cache_dates=True)


In [76]:
data = pd.merge(price_data, tweets_data, left_index=True, right_index=True)

# Market Cap = Market Price × Circulating Supply
del data['dogecoin_market_cap']

# these attributes negatively affects accuracy
del data['bitcoin_market_cap']

# mark all missing values
data.replace('?', nan, inplace=True)

# make the dataset numeric
data = data.astype('float32')

fill_missing(data.values)

data.head()


,dogecoin_price,dogecoin_volume,bitcoin_price,bitcoin_volume,tweets_count,usernames_count,likes_sum,replies_sum,retweets_sum,compound_score_sum,positive_score_sum,negative_score_sum,neutral_score_sum,likes_mean,replies_mean,retweets_mean,compound_score_mean,positive_score_mean,negative_score_mean,neutral_score_mean
timestamp,,,,,,,,,,,,,,,,,,,,
2021-01-01 06:30:00,0.004653,125931232.0,29018.490234,3.778730e+10,25.0,25.0,34.0,8.0,1.0,4.7231,3.607,0.769,24.624001,1.172414,0.275862,0.034483,0.162866,0.124379,0.026517,0.849103
2021-01-01 07:30:00,0.004732,120294400.0,29347.054688,3.804020e+10,24.0,24.0,56.0,10.0,71.0,5.5557,3.479,0.427,22.094000,2.153846,0.384615,2.730769,0.213681,0.133808,0.016423,0.849769
2021-01-01 08:30:00,0.004757,117695288.0,29261.318359,3.908687e+10,23.0,23.0,41.0,7.0,8.0,4.1285,2.573,0.746,22.681000,1.576923,0.269231,0.307692,0.158788,0.098962,0.028692,0.872346
2021-01-01 09:30:00,0.004929,125960880.0,29324.677734,3.896300e+10,39.0,39.0,47.0,8.0,19.0,5.9942,4.390,0.814,37.796001,1.093023,0.186047,0.441860,0.139400,0.102093,0.018930,0.878977
2021-01-01 10:30:00,0.004889,133725928.0,29169.453125,3.873581e+10,32.0,32.0,279.0,27.0,65.0,9.1851,6.805,1.820,26.378000,7.971428,0.771429,1.857143,0.262431,0.194429,0.052000,0.753657


Save the dataset to reduce processing time

In [77]:
create_folder('./processed_datasets')
data.to_csv('./processed_datasets/dogecoin_price_sentiments.csv')


Resample hourly data to total for each day

In [78]:
# load the new file
dataset = pd.read_csv('./processed_datasets/dogecoin_price_sentiments.csv', header=0,
                      infer_datetime_format=True, parse_dates=['timestamp'], index_col=['timestamp'])
dataset_resampled = dataset.reindex(dataset.index.union(
    dataset.resample('1H').asfreq().index)).interpolate().resample('1H').asfreq()

# summarize
print(dataset_resampled.shape)
print(dataset_resampled.head())

create_folder('./processed_datasets')
dataset_resampled.to_csv(
    './processed_datasets/dogecoin_price_sentiments_hours.csv')


(2832, 20)
                     dogecoin_price  ...  neutral_score_mean
timestamp                            ...                    
2021-01-01 06:00:00             NaN  ...                 NaN
2021-01-01 07:00:00        0.004693  ...            0.849436
2021-01-01 08:00:00        0.004745  ...            0.861058
2021-01-01 09:00:00        0.004843  ...            0.875661
2021-01-01 10:00:00        0.004909  ...            0.816317

[5 rows x 20 columns]
